In [44]:
import numpy as np
import matplotlib.pyplot as plt

import pygimli as pg
from pygimli.viewer.mpl import drawModel1D
import pandas as pd

import sys, os

# Disable
def blockPrint():
    sys.stdout = open(os.devnull, 'w')

# Restore
def enablePrint():
    sys.stdout = sys.__stdout__

In [45]:
df = pd.read_csv("cowsns.csv")
ab2 = df["AB/2 [m]"]
rhoa = df["R [rho]"]

In [46]:
# nlay = 20 # number of layers
# lam = 200.  # (initial) regularization parameter
errPerc = 3.  # relative error of 3 percent=
# ab2 = df["AB/2 [m]"]
# rhoa = df["R [rho]"]
# mn2 = ab2 / 3.  # MN/2 distance (potential electrodes)


def invert(nlay, ab2, mn2, lam, rhoa, mscheme=0.5):
    print(2*max(rhoa))
   
    f = pg.core.DC1dModelling(nlay, ab2, mn2)

    transThk = pg.trans.TransLog() 
    transRho = pg.trans.TransLogLU(1, 2*max(rhoa)) 
    transRhoa = pg.trans.TransLog() 

    f.region(0).setTransModel(transThk)  
    f.region(1).setTransModel(transRho)  

    paraDepth = max(ab2) / 3.  # rule-of-thumb for Wenner/Schlumberger
    f.region(0).setStartValue(paraDepth / nlay / 2)
    f.region(1).setStartValue(np.median(rhoa))

    inv = pg.core.Inversion(rhoa, f, transRhoa, True) 
    inv.setRelativeError(errPerc / 100.0)  
    inv.setLambda(lam) 
    inv.setMarquardtScheme(mscheme)
    #help(inv)
    print("starting lambda value:", inv.getLambda(), lam)
    model = f.createStartVector()  
    #model[nlay] *= 1.5
    inv.setModel(model)
    
    blockPrint()
    model = inv.run()
    enablePrint()
    
    res, thk = model[nlay-1:nlay*2-1], model[0:nlay-1]
    print('rrms={:.2f}%, chi^2={:.3f}'.format(inv.relrms(), inv.chi2()))

    fig, ax = plt.subplots(ncols=2, figsize=(8, 6))  # two-column figure
   
    drawModel1D(ax[0], thk, res, color='b', label=f"chi2 = {round(inv.chi2())}")
    ax[0].grid(True, which='both')
    ax[0].set_ylabel('z (m)')
    ax[0].set_xlabel(r'$\rho$ ($\Omega$m)')
    ax[0].legend(loc='best')
    #ax[0].set_ylim(max(ab2), 0)
  
    ax[1].loglog(np.array(rhoa).reshape(len(rhoa),1), np.array(ab2).reshape(len(ab2),1), 'rx-', label='measured')
    ax[1].loglog(np.array(inv.response()).reshape(len(inv.response()),1), ab2, 'b-', label='fitted')

    ax[1].set_ylim((max(ab2), min(ab2)))
    ax[1].grid(True, which='both')
    ax[1].set_xlabel(r'$\rho_a$ ($\Omega$m)')
    ax[1].set_ylabel('AB/2 (m)')
    ax[1].legend(loc='best')
    
    fig.suptitle(f"VES Inversion Best Fit with {nlay} Layers")
    plt.show()
    
    return model, inv.chi2()

#model, chi2 = invert(len(rhoa)+1, ab2, mn2, lam, rhoa, 0.4)

In [47]:
df

,AB/2 [m],R [rho]
0,1.0,888.285323
1,1.5,1143.539726
2,2.0,1353.633735
3,3.0,1621.847207
4,4.0,1608.102740
5,5.0,1611.849089
6,6.0,1306.187832
7,7.0,1101.167495
8,8.0,941.299699
9,9.0,816.861214


In [48]:
#todo: check that the change in depths for both graphs is because the resistivity is constant after the last layer

In [49]:
# Load the rest of the data

import csv

rows = []
with open('cowsjackpot.csv', newline='') as csvfile:
     spamreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
     for row in spamreader:
         rows.append(', '.join(row))

In [50]:
import io
colnames=["N", "a", "n", "ab2", "r", "rhoa", "sigma"] 
dcowsns = pd.read_csv(io.StringIO('\n'.join(rows[2:14])), names=colnames)
dcowses = pd.read_csv(io.StringIO('\n'.join(rows[18:30])), names=colnames)
mcowsns = pd.read_csv(io.StringIO('\n'.join(rows[34:47])), names=colnames)

djackns = pd.read_csv(io.StringIO('\n'.join(rows[51:64])), names=colnames)
mjackns = pd.read_csv(io.StringIO('\n'.join(rows[68:])), names=colnames)

In [53]:
# Try to find the best fit for each dataset seperately and then average the dipole for cows and repeat.
mjackns
np.zeros(ab2.shape) + 0.5
ab2

0      1.0
1      1.5
2      2.0
3      3.0
4      4.0
5      5.0
6      6.0
7      7.0
8      8.0
9      9.0
10    10.0
Name: ab2, dtype: float64

In [52]:
df2 = dcowsns


enablePrint()
print(ab2)
ab2 = df2["ab2"][:-1]
mn2 = np.zeros(ab2.shape) + 0.5
rhoa = df2["rhoa"][:-1]

nlay = 2

model, chi2 = invert(nlay, ab2, mn2, 200, rhoa, 0.9)
res, thk = model[nlay-1:nlay*2-1], list(model[0:nlay-1])
thk.append(np.inf)
layers = pd.DataFrame({"Resistivity (Ohm*m)": res, "Conductivity (S/m)": 1/res, "Thickness (m)": thk})
layers

/opt/anaconda3/conda-bld/pgcore_1645178403028/work/gimli/core/src/regionManager.h:182		void GIMLI::Region::setStartValue(double)  is deprecated 
/opt/anaconda3/conda-bld/pgcore_1645178403028/work/gimli/core/src/regionManager.h:182		void GIMLI::Region::setStartValue(double)  is deprecated 
/opt/anaconda3/conda-bld/pgcore_1645178403028/work/gimli/core/src/modellingbase.cpp:523		GIMLI::RVector GIMLI::ModellingBase::createStartVector()  is deprecated 


RuntimeError: /opt/anaconda3/conda-bld/pgcore_1645178403028/work/gimli/core/src/inversion.cpp:95		double GIMLI::RInversion::getPhiD(const GIMLI::RInversion::Vec &) const  getPhiD == inf

In [10]:
n = df2["n"]
a = df2["a"]

In [29]:
mx = -a/2
my = 0

nx = a/2
ny = 0

ax = -(n+1/2)*a
ay = 0

bx = (n+1/2)*a
by = 0

In [30]:
k = 2*np.pi/(((ax-mx)**2 + (ay-my)**2)**-0.5 - ((bx-mx)**2 + (by-my)**2)**-0.5 - ((ax-nx)**2 + (ay-ny)**2)**-0.5 + ((bx-nx)**2 + (by-ny)**2)**-0.5)

In [39]:
AB2 = abs((ax-bx)/2)
MN2 = abs((mx-nx)/2)

0     0.5
1     0.5
2     0.5
3     0.5
4     0.5
5     0.5
6     0.5
7     0.5
8     0.5
9     0.5
10    0.5
11    0.5
Name: a, dtype: float64

In [31]:
k*df2["r"]

0     302.535373
1     392.070763
2     440.608370
3     488.753277
4     490.347635
5     431.537021
6     390.845542
7     231.260489
8     196.271001
9     104.010279
10      0.000000
11      0.000000
dtype: float64

In [26]:
df2["rhoa"]

0     302.535372
1     392.070763
2     440.608370
3     488.753277
4     490.347635
5     431.537021
6     390.845542
7     231.260489
8     196.271001
9     104.010279
10      0.000000
11      0.000000
Name: rhoa, dtype: float64

In [15]:
k

0       6.283185
1      11.780972
2      18.849556
3      37.699112
4      62.831853
5      94.247780
6     131.946891
7     175.929189
8     226.194671
9     282.743339
10    345.575192
11    414.690230
dtype: float64

In [28]:
3.1415*df2["a"]*df2["n"]*(df2["n"]+1)

0       2.356125
1       6.283000
2      11.780625
3      27.488125
4      49.478625
5      77.752125
6     112.308625
7     153.148125
8     200.270625
9     253.676125
10    313.364625
11    379.336125
dtype: float64

In [21]:
k = 3.1415*2*(n**2 - 1 + (n**2/a**2)*(2*a+1))
k

0        0.000
1       18.849
2       50.264
3      150.792
4      301.584
5      502.640
6      753.960
7     1055.544
8     1407.392
9     1809.504
10    2261.880
11    2764.520
dtype: float64